In [1]:
import dime
import numpy as np

In [2]:
nobs = 14
nk = 125
dime.setup(nobs*nk, nk)

In [3]:
j

NameError: name 'j' is not defined

In [ ]:
class Dime:
    """A class for running Maximum Entropy inversions of Doppler images."""

    def __init__(self, data, weights, Rmatrix, alpha, nk):
        self.data = data
        self.weights = weights
        self.Rmatrix = Rmatrix
        self.alpha = alpha
        self.nk = nk
        self.setup()